In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from time import time
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [2]:
np.random.seed(42)
train_clicks=pd.read_csv('raw_data/train.csv',dtype={'id':'str'})
test_clicks=pd.read_csv('raw_data/test.csv',dtype={'id':'str'})

In [3]:
def df_to_ffm(x,y,file,cate_cols):
    x.index=np.arange(x.shape[0])
    y.index=np.arange(x.shape[0])
    cate_idx=set([-1])
    for i in range(x.shape[1]):
        if x.columns[i] in cate_cols:
            cate_idx.add(i)
            
    with open(file,'w') as f:
        for i in range(x.shape[0]):
            s=str(y[i])
            
            for j in range(x.shape[1]):
                if j in cate_idx:
                    s += ' %d:%d:1' % (j,x.values[i,j])
                else:
                    s += ' %d:%s' % (j,x.values[i,j])
                    
            f.write(s+'\n')
        
        
def glabel_encode(df,cate_cols):
    if len(cate_cols) > 0:
        for c in cate_cols:
            l=LabelEncoder()
            df[c]=df[c].astype('str')
            df[c]=l.fit_transform(df[c])
        
df=pd.DataFrame({'a':np.arange(5),'b':['a','b','a','c','b'],'c':['b','a','d','c','a'],'d':np.arange(5),'e':np.arange(5)})
glabel_encode(df,['b','c','d'])
df_to_ffm(df.drop(['e'],axis=1),df.e,'ffm_data/test.o',['b','c','d'])

In [4]:
t=time()
traintst=pd.concat([train_clicks,test_clicks])
traintst['time']=traintst.hour
traintst['hour']=traintst.time%100
traintst['day']=(traintst.time-traintst.hour)//100%100

cate_cols=train_clicks.columns.difference(['id','click','device_id','device_ip','day','time'])
glabel_encode(traintst,cate_cols)

train_c=traintst.iloc[:train_clicks.shape[0]]
df_to_ffm(train_c.drop(['id','click','device_id','device_ip','time','day'],axis=1),train_c.click,'ffm_data/train.txt',cate_cols)
test_c=traintst.iloc[train_clicks.shape[0]:]
df_to_ffm(test_c.drop(['id','click','device_id','device_ip','time','day'],axis=1),test_c.click.fillna(0),'ffm_data/test.txt',cate_cols)
    
print('time cost %ds'%int(time()-t))


time cost 18212s


In [ ]:
import xlearn as xl

fm_model = xl.create_ffm()
fm_model.setSigmoid()
fm_model.setTrain("ffm_data/train.txt")  # Training data
fm_model.setTest('ffm_data/test.txt')
# fm_model.setValidate("./small_test.txt")  # Validation data

# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
param = {'task':'binary','epoch':50}

# Start to train
# The trained model will be stored in model.out
fm_model.fit(param, 'ffm_data/model.out')
fm_model.predict('ffm_data/model.out','ffm_data/output.txt')

In [3]:
test_clicks=pd.read_csv('raw_data/test.csv',dtype={'id':'str'},usecols=['id'])
outputs = pd.read_csv('ffm_data/output_1.txt',header=None)

In [4]:
test_clicks.shape,outputs.shape

((4577464, 1), (4577464, 1))

In [5]:
test_clicks['click']=outputs
test_clicks[['id','click']].to_csv('output/ffm.csv',index=False)